# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'C:/Users/user/Documents/GitHub/2nd-ML-100Days/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']#目標
ids = df_test['PassengerId']#ID
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])#預處理所以先合併
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [4]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

目前看來df.min()+df.max())/2最好，但不知該怎麼解釋，而且可能跟cv分組有關，有稍微調過cv，不同cv時準確最高者會不一樣，因此值得思考設計使用的補植方法與cv設定

In [84]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6960299128976762

In [85]:
#空缺值補平均值，邏輯斯回規
df_m2=df.fillna(df.mean())
train_X = df_m2[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6981761033723469

In [89]:
#'''
#print(type(df.mode()))
#print(df.mode())
#df.mode()
#'''

In [87]:
#a.iloc[0]#Dataframe to Series

In [88]:
#'''
#print(type(df.mean()))
#df.mean()
#'''

In [90]:
#眾數，型態需轉換
df_m5=df.fillna(df.mode().iloc[0])
train_X = df_m5[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6936879984197388

In [91]:
df_m3=df.fillna(0)
train_X = df_m3[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971535084032942

In [92]:
df_m4=df.fillna(-100)
train_X = df_m4[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6814104044028857

In [93]:
df_m6=df.fillna(df.max())
train_X = df_m6[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6960173587579487

In [113]:
df_m7=df.fillna(df.min())
train_X = df_m7[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971535084032942

In [114]:
df_m7=df.fillna((df.min()+df.max())/2)
train_X = df_m7[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6993123239450355

In [80]:
df.mean()

Pclass     2.294882
Age       29.881138
SibSp      0.498854
Parch      0.385027
Fare      33.295479
dtype: float64

In [81]:
df.mode().iloc[0]

Pclass     3.00
Age       24.00
SibSp      0.00
Parch      0.00
Fare       8.05
Name: 0, dtype: float64

In [79]:
(df.min()+df.max())/2

Pclass      2.0000
Age        40.0850
SibSp       4.0000
Parch       4.5000
Fare      256.1646
dtype: float64

In [82]:
df.max()

Pclass      3.0000
Age        80.0000
SibSp       8.0000
Parch       9.0000
Fare      512.3292
dtype: float64

In [83]:
df.min()

Pclass    1.00
Age       0.17
SibSp     0.00
Parch     0.00
Fare      0.00
dtype: float64

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [119]:
"""
Your Code Here
"""
df = df.fillna(-1)#每補Nan值時會跑出下面錯誤
'''
Input contains NaN, infinity or a value too large for dtype('float64').'''
df_temp = MinMaxScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X,train_Y,cv=5).mean()

0.6971346062663598

In [120]:
df_temp = StandardScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X,train_Y,cv=5).mean()

0.6982582017719778

In [121]:
train_X = df[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X,train_Y,cv=5).mean()

0.6960299128976762

In [ ]:
'''
機器學習與統計的最大差別, 在於機器學習本身是沒有"標準差"這種概念的

所以並不像統計一樣, 有95信心水準, 有虛無假設, 有p值小於0.05為顯著這種概念

所以差多少才足夠好, 沒有通用的檢驗標準, 

完全取決於資料集與你選用的模型而定
'''